In [ ]:
# dataframe manipulations
import pandas as pd

#math
from scipy.sparse import csr_matrix
import numpy as np

#images
from IPython.display import display
from PIL import Image


#Data viz
import matplotlib.pyplot as plt
import seaborn as sns

#models
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity 


In [ ]:
book_base = pd.read_csv('data/Books.csv',on_bad_lines='error',encoding='latin-1')
user_base = pd.read_csv('data/Users.csv',on_bad_lines='error',encoding='latin-1')
rate_base = pd.read_csv('data/Ratings.csv',on_bad_lines='error',encoding='latin-1')

## DA

In [ ]:
print(f'book possui {book_base.shape[0]} linhas e {book_base.shape[1]} colunas')
print(f'user possui {user_base.shape[0]} linhas e {user_base.shape[1]} colunas')
print(f'rate possui {rate_base.shape[0]} linhas e {rate_base.shape[1]} colunas')

### Books dataset

In [ ]:
book_base.head(2)

Vamos retirar algumas colunas do dataset `book_base`


In [ ]:
books = book_base.drop(columns=['Image-URL-S', 'Image-URL-M'])

In [ ]:
books.head(2)

Para facilitar a manipulação dos dados, irei renomear algumas colunas

In [ ]:
books = books.rename(columns={
    'Book-Title' : 'title',
    'Book-Author': 'author',
    'Year-Of-Publication': 'year',
    'Publisher': 'publisher'   
    
})

In [ ]:
books.head(2)

In [ ]:
books.info()

### User dataset

In [ ]:
user_base.info()

In [ ]:
user_base.shape

In [ ]:
user = user_base

In [ ]:
user.columns

In [ ]:
user = user.rename(columns={
    'User-ID':'user-id',
    'Location':'location',
    'Age':'age'
    
})

In [ ]:
user.head(2)

### Rating

In [ ]:
rate_base.shape

In [ ]:
rate_base.info()

In [ ]:
rate_base['User-ID'] = rate_base['User-ID'].astype('object')

In [ ]:
rating= rate_base

In [ ]:
rating['Book-Rating'].value_counts().sort_values(ascending = False)

In [ ]:
rating = rating.rename(columns={
    'User-ID':'user-id',
    'Book-Rating':'rating'
    
})

###  Top 5 usuários que mais avaliaram Quantidade de avaliações por usuário

In [ ]:
rating['user-id'].value_counts()[:5]


In [ ]:
top5 = rating['user-id'].value_counts()[:5].reset_index().rename(columns = {'index':'number_review'})

In [ ]:
top5 = top5.sort_values(by = 'number_review', ascending = False);top5

In [ ]:
sns.barplot(data = top5, x = 'user-id', y = 'number_review')

## Filtragem de atributos

Neste momento faremos uma filtragem nos dados, com fito de facilitar a implementação do algorítimo adiante

Como vimos acima, na sessão em que manipulei o dataset `Rating`, temos muitos usuários com poucas avaliações, e isto pode nos gerar um problema no nosso sistema de recomendação. Por tanto, como decisão de negócio iremos filtrar estes clientes 

### Queremos apenas os usuários com <b>mais de 200 avaliações</b>

In [ ]:
x = rating['user-id'].value_counts() > 200 ; x

In [ ]:
# x em um dataframe
x = x.reset_index()
#selecionando apenas as colunas verdadeiras
x = x.loc[x['user-id'] == True];
# lista de usuários
user_idx = x['index'].tolist()

In [ ]:
ratings = rating[rating['user-id'].isin(user_idx)];
ratings

Conseguimos os usuários que se enquadram nesta decisão de negócio

### Apartir disso, temos como descobrir os nomes dos livros avaliados por cada usuário, a partir de seu identificador `ISBN`

In [ ]:
ratings_books = ratings.merge(books,on = 'ISBN'); ratings_books.head()

In [ ]:
ratings_books.shape

## Livro x Rating


In [ ]:
#agrupando Livros x Rating
num_rating = ratings_books.groupby('title')['rating'].count().reset_index();
#renomeando a coluna Rating para num_rating
num_rating.rename(columns={'rating':'num_ratings'},inplace = True);num_rating

In [ ]:
# Titulos com mais de 50 avaliações
num_rating_over50 = num_rating.sort_values(by='num_ratings',ascending=False)
num_rating_over50 = num_rating_over50.loc[num_rating_over50['num_ratings']>50]; num_rating_over50

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data = num_rating_over50[:5], x = 'title', y = 'num_ratings')
plt.title('top 5 livros mais avaliados')
plt.xticks(rotation = 45)

Com este novo atributo `num_ratings` vamos adiciona-lo à tabela de `rating_books` e formar uma tabela completa das features necessárias para o modelo

In [ ]:
final_rating = ratings_books.merge(num_rating_over50, on='title'); final_rating

In [ ]:
final_rating.drop_duplicates(['user-id','title'],inplace=True); final_rating.shape

## Filtro colaborativo

### TEXTO FITLRO COLABORATIVO

**Como Funciona um Sistema de Recomendação**
https://www.youtube.com/live/mtRIGq4nSu4?feature=share&t=721

In [ ]:
img = Image.open('img/ey-bm-recomendacao1v2.jpg');img

In [ ]:
Fonte: https://www.ey.com/pt_br/strategy-transactions/ey-business-modeling-data-analytics/modelo-de-recomendacao-de-produtos1

### Matriz de Usuário x Item

In [ ]:
img2 = Image.open('img/usuario_item.png');img2


Fonte: https://www.sidi.org.br/introducao-aos-sistemas-de-recomendacao/

In [ ]:
book_pivot = final_rating.pivot_table(columns='user-id', index= 'title', values= 'rating' );
book_pivot

Observamos uma grande quantidade de valores `NAN`, pois diversos usuários não tiveram a oportunidade de ler alguns livros

Decidi definir a ausência de avaliação como nota 0

In [ ]:
book_pivot.fillna(0, inplace=True);book_pivot

Em casos onde temos matrizes com uma elevada quantidade de valores 0, o ideal é transforma-la em uma matriz esparsa, a fim de melhorar o tempo de execução e armazenamento

https://acervolima.com/como-criar-uma-matriz-esparsa-em-python/

In [ ]:
book_user = csr_matrix(book_pivot);
book_user

# Modelagem 

In [4]:
model = NearestNeighbors(algorithm='brute')
model.fit(book_user)

NameError: name 'NearestNeighbors' is not defined

Este algorítimo retorna 2 valores, distância e sugestão

# Predições
Passamos para o `model.kneighbors()` o elemento que queremos analizar identificado pela linha

## NearestNeighbors 

In [5]:
titulo = 0 # linha na tabela `book_pivot` que representa o título, neste caso o livro 1984

In [ ]:
distance,suggestions = model.kneighbors(book_pivot.iloc[titulo,:].values.reshape(1,-1))

In [ ]:
for i in range(len(suggestions)):
    print(f'Para o livro {(book_pivot.index[suggestions[i]])[0]}: você também vai gostar de : {list(book_pivot.index[suggestions[i]][1:])}')

In [ ]:
def recommend_nn(book_name):
    index = np.where(book_pivot.index==book_name)[0][0] #obtendo o nome do livro
    distance , suggestions = model.kneighbors(book_pivot.iloc[index,:].values.reshape(1,-1),n_neighbors=6) #6 vizinhos mais próximos
    
    for i in range(len(suggestions)):
        books = book_pivot.index[suggestions[i]]
        print(f'Você leu: {book_name}, também vai gostar de:')
        for j in books[1:]:
            print(j)

In [ ]:
recommend_nn('The Da Vinci Code')

## Similaridade dos cosenos

In [ ]:
similarity_score = cosine_similarity(book_user)

In [ ]:
def recommend_sc(book_name):
    index = np.where(book_pivot.index==book_name)[0][0]
    similar_books = sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1], reverse=True)[1:6]
    
    data = []
    
    for i in similar_books:
        item = []
        temp_df = books[books['title'] == book_pivot.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('title')['title'].values))
        item.extend(list(temp_df.drop_duplicates('title')['author'].values))
        item.extend(list(temp_df.drop_duplicates('title')['Image-URL-L'].values))
        
        data.append(item)
        #for j in data:
         #   print(j)
    msg = f'Você leu {book_name}, também vai gostar de:'
        
        
    return msg, data

In [ ]:
recommend_sc('The Da Vinci Code')

# Persistência do modelo


In [6]:
book_name = book_pivot.index

NameError: name 'book_pivot' is not defined

In [ ]:
import pickle
pickle.dump(model,open('models/model.pkl','wb'))
pickle.dump(book_name,open('models/book_name.pkl','wb'))
pickle.dump(book_pivot,open('models/book_pivot.pkl','wb'))
pickle.dump(final_rating,open('models/final_rating.pkl','wb'))
